# langGraph Practice 1



![구성도](/home/ansgyqja/AI_application/images/9.structure_practice_1.png)

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("langGraph_practice_1")

LangSmith 추적을 시작합니다.
[프로젝트명]
langGraph_practice_1


In [3]:
from typing import Annotated,TypedDict,List,Literal
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langchain_teddynote.evaluator import GroundednessChecker
from langchain import hub
from operator import itemgetter
from langchain_teddynote.messages import messages_to_history
from langchain_teddynote.tools.tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('get_naver_news'), '..')))
from module.utils import re_write_prompt,get_gemini,format_docs
from module.get_local_pdf import rag_pdf

In [4]:
class State(TypedDict):
    question: Annotated[list, add_messages]  # 질문(누적되는 list)
    context : Annotated[str,'Context']
    answer : Annotated[str,'Answer']
    relevant : Annotated[str,'yes or no']
    chat_history:Annotated[list,add_messages]

# re write node
def re_write_node(state : State) -> State:
    chain = re_write_prompt | get_gemini() | StrOutputParser()
    response = chain.invoke({"question": state['question']})
    return State({"question":response})

# retriever node
def retriever_node(state : State) -> State:
    retriever =rag_pdf().get_retriever()
    latest_question = state["question"][-1].content
    document = retriever.invoke(latest_question)
    context = format_docs(document)
    return State({"context":context})

def retriever_relevant(state : State):
    question_answer_relevant = GroundednessChecker(
        llm=get_gemini(), target="question-retrieval"
    ).create()
    response = question_answer_relevant.invoke(
        {"question": state["question"][-1].content, "context": state["context"]}
    )
    return State({"relevant":response.score})
    

def llm_answer_node(state: State):
    prompt = hub.pull("teddynote/rag-prompt-chat-history")
    chain =(
        {
            "question": itemgetter("question"),
            "context": itemgetter("context"),
            "chat_history": itemgetter("chat_history"),
        } | 
        prompt | get_gemini() | StrOutputParser()
        )
    answer = chain.invoke(
       {
            "question": state['question'][-1].content,
            "context": state['context'],
            # "chat_history": ""
            "chat_history": messages_to_history(state['chat_history']),  # 문자열 리스트로 변환
        }
    )
    return {
            "answer": answer,
            "chat_history":[("user" ,state['question'][-1].content),("assistant" , answer)]
            }

def web_node(state:State):
    web_search = TavilySearch()
    search_context = web_search.search(query=state['question'])
    return State({"context":search_context})

def search_relevant(state : State):
    retrieval_answer_relevant = GroundednessChecker(
        llm=get_gemini(), target="retrieval-answer"
    ).create()
    response = retrieval_answer_relevant.invoke(
        {"question": state["question"][-1].content, "context": state["context"]}
    )
    return State({"relevant":response.score})
   
def is_retriever(state : State)-> Literal["llm_answer_node","web_node"]:
    is_relevant = state.get("relevant","no")
    if is_relevant == "yes":
        return "llm_answer_node"
    return "web_node"

def is_search(state : State)-> Literal["llm_answer_node","re_write_node"]:
    is_relevant = state.get("relevant","no")
    if is_relevant == "yes":
        return "llm_answer_node"
    return "re_write_node"


In [5]:
state_graph = StateGraph(State)
state_graph.add_node('re_write_node',re_write_node)
state_graph.add_node('retriever_node',retriever_node)
state_graph.add_node('retriever_relevant',retriever_relevant)
state_graph.add_node('llm_answer_node',llm_answer_node)
state_graph.add_node('web_node',web_node)
state_graph.add_node('search_relevant',search_relevant)


state_graph.add_edge(START,'re_write_node')
state_graph.add_edge('re_write_node','retriever_node')
state_graph.add_edge('retriever_node','retriever_relevant')
state_graph.add_conditional_edges(
    source='retriever_relevant',
    path=is_retriever,
    path_map={
        "llm_answer_node": "llm_answer_node",  # 관련성이 있으면 답변을 생성합니다.
        "web_node": "web_node",  # 관련성이 없으면 다시 검색합니다.
    },
)
state_graph.add_edge('web_node','search_relevant')
state_graph.add_conditional_edges(
    source='search_relevant',
    path=is_search,
    path_map={
        "llm_answer_node": "llm_answer_node",  # 관련성이 있으면 답변을 생성합니다.
        "re_write_node": "re_write_node",  # 관련성이 없으면 다시 검색합니다.
    },
)
state_graph.add_edge('llm_answer_node',END)



In [6]:
# 메모리 저장소 생성
memory = MemorySaver()
graph = state_graph.compile(checkpointer=memory)
mermaid_code = graph.get_graph().draw_mermaid()
print(mermaid_code)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	re_write_node(re_write_node)
	retriever_node(retriever_node)
	retriever_relevant(retriever_relevant)
	llm_answer_node(llm_answer_node)
	web_node(web_node)
	search_relevant(search_relevant)
	__end__([<p>__end__</p>]):::last
	__start__ --> re_write_node;
	re_write_node --> retriever_node;
	retriever_node --> retriever_relevant;
	retriever_relevant -.-> llm_answer_node;
	retriever_relevant -.-> web_node;
	search_relevant -.-> llm_answer_node;
	search_relevant -.-> re_write_node;
	web_node --> search_relevant;
	llm_answer_node --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [7]:
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph, invoke_graph, random_uuid

# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})

# 질문 입력
inputs = State({'question':'엔트로피 투자금액'})

# for event in graph.stream({'question':'엔트로피 투자금액'},config=config,stream_mode="values"):
#     print(event)
# 그래프 실행
stream_graph(graph, inputs, config)


🔄 Node: re_write_node 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
엔트로피의 투자 금액은 얼마인가요?
🔄 Node: retriever_relevant 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: llm_answer_node 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침입니다.
- ../data/SPRI_AI_Brief_2023년12월호_F.pdf (14쪽)

In [8]:
# 최종 출력

outputs = graph.get_state(config).values
print(outputs)
# print(f'Original Question: {outputs["question"][0].content}')
# print(f'Rewritten Question: {outputs["question"][-1].content}')
# print("===" * 20)
# print(f'Answer:\n{outputs["answer"]}')

{'question': [HumanMessage(content='엔트로피 투자금액', additional_kwargs={}, response_metadata={}, id='60e0ac60-be6b-4449-8b90-9fd1357ce7f6'), HumanMessage(content='엔트로피의 투자 금액은 얼마인가요?', additional_kwargs={}, response_metadata={}, id='e80114b3-6fb4-44a2-9546-05e9f3523f67')], 'context': '<document><content>1. 정책/법제  2. 기업/산업 3. 기술/연구  4. 인력/교육\n미국 프런티어 모델 포럼, 1,000 만 달러 규모의 AI 안전 기금 조성\nn구글, 앤스로픽 , 마이크로소프트 , 오픈AI가 참여하는 프런티어 모델 포럼이 자선단체와 함께 AI \n안전 연구를 위한 1,000 만 달러 규모의 AI 안전 기금을 조성\nn프런티어 모델 포럼은 AI 모델의 취약점을 발견하고 검증하는 레드팀 활동을 지원하기 위한 \n모델 평가 기법 개발에 자금을 중점 지원할 계획KEY Contents\n£프런티어 모델 포럼, 자선단체와 함께 AI 안전 연구를 위한 기금 조성\nn구글, 앤스로픽 , 마이크로소프트 , 오픈AI가 출범한 프런티어 모델 포럼이 2023년 10월 25일 AI 안전 \n연구를 위한 기금을 조성한다고 발표\n∙참여사들은 맥거번 재단(Patrick J. McGovern Foundation), 데이비드 앤 루실 패커드 재단(The \nDavid and Lucile Packard Foundation ) 등의 자선단체와 함께 AI 안전 연구를 위한 기금에 \n1,000 만 달러 이상을 기부 \n∙또한 신기술의 거버넌스와 안전 분야에서 전문성을 갖춘 브루킹스 연구소 출신의 크리스 메서롤 (Chris \nMeserole) 을 포럼의 상무이사로 임명\nn최근 AI 기술이 급속히 발전하면서 AI 안전에 관한 연구가 부족한 시점에 , 포럼은 이러한